# Use Machine Learning Technique to Predict pm2.5
Steps
1. Load csv data
2. Clean data and reorganize data
3. Find the best linear regression with training set, gradient, Adagrad
4. Forecast pm2.5

In [17]:
import csv
import pandas as pd
import chardet
import numpy as np

from numpy.linalg import inv
import random
import math
import sys

# Step 1: Load Data from csv

In [18]:
# [ My way ]
# Careful to read because raw data has string and number
with open('train.csv', 'rb') as f:
    train = chardet.detect(f.read())  # or readline if the file is large

# this will show all the data look like
# pd.read_csv('train.csv', encoding=train['encoding'])
traindata = pd.read_csv('train.csv', encoding=train['encoding'])
# see what's the data look like for first few rows
traindata.head()

,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5


In [19]:
temp = traindata.values
print(temp)
# the problem is everything is in "string" "15"

[['2014/1/1' '豐原' 'AMB_TEMP' ..., '15' '15' '15']
 ['2014/1/1' '豐原' 'CH4' ..., '1.8' '1.8' '1.8']
 ['2014/1/1' '豐原' 'CO' ..., '0.35' '0.36' '0.32']
 ..., 
 ['2014/12/20' '豐原' 'WIND_DIREC' ..., '118' '100' '105']
 ['2014/12/20' '豐原' 'WIND_SPEED' ..., '1.5' '2' '2']
 ['2014/12/20' '豐原' 'WS_HR' ..., '1.6' '1.8' '2']]


## [ TA way ]
1. data = []
2. 每一個維度儲存一種污染物的資訊
3. for i in range(18):
4. data.append([])

In [20]:
# [ TA way ]
data = []
# 每一個維度儲存一種污染物的資訊
for i in range(18):
	data.append([])

In [21]:
text = open('train.csv', 'r', encoding='big5') 
row = csv.reader(text , delimiter=",")
n_row = 0
for r in row:
    # 第0列沒有資料
    if n_row != 0:
        # 每一列只有第3-27格有值(1天內24小時的數值)
        for i in range(3,27):
            if r[i] != "NR":
                data[(n_row-1)%18].append(float(r[i]))
            else:
                data[(n_row-1)%18].append(float(0))	
    n_row = n_row+1
text.close()
print("data_shape : " , np.shape(data))

data_shape :  (18, 5760)


# Step 2: Parse Data: How to extract feature 
(Pseudo code)
1. 宣告train_x儲存前9小時data，以及train_y紀錄第十小時pm2.5值
2. for i =1月、2月......
3. 	 取樣每連續10個小時：
4. 		train_x.append(前9小時所有data)
5. 		train_y.append(第10小時pm2.5值)
6. 在train_x每筆data中加入bias

In [22]:
train_x = []
train_y = [] 
# 因為一個月取前二十天當training data所以要以月份為loop
for m in range(12):
    # 20*24=480 480-9=471一個月的資料數
    for h in range(471):
        train_y.append(data[9][480*m+h+9])
        train_x.append([])
        # 18種污染物
        for i in range(18):
            # 9個小時
            for t in range(9):
                train_x[471*m+h].append(data[i][m*480+h+t])
print("train_y_shape : " , np.shape(train_y))
print("train_x_shape : " , np.shape(train_x))
type(train_y)
print("train_x_type : " , type(train_x) )

train_x = np.array(train_x)
train_y = np.array(train_y)
print("train_y_shape : " , np.shape(train_y))
print("train_x_shape : " , np.shape(train_x))
type(train_y)
print("train_x_type : " , type(train_x) )
print("train_y : " , train_y)

train_y_shape :  (5652,)
train_x_shape :  (5652, 162)
train_x_type :  <class 'list'>
train_y_shape :  (5652,)
train_x_shape :  (5652, 162)
train_x_type :  <class 'numpy.ndarray'>
train_y :  [ 30.  41.  44. ...,  17.  24.  29.]


# Step 3: Linear Regression

## Original
* X = Dim( #sample , #parameters )
* w = Dim( #parameters , 1 )
* b = scaler
* Y = Xw + b
* Y = Dim( #sample , 1 )

## Rewrite bias into weight vector
* X = Dim( #sample , #parameters + 1 )
* w = Dim( #parameters+1 , 1 )
* Y = Dim( #sample , 1 )

## gradient descent
* gradient = -2 * x * (Y - Yhat)

## (Pseudo code)
1. 宣告weight vector、初始learning rate、# of iteration
2. for i_th iteration :
3. 	 y’ = train_x 和 weight vector 的 內積
4. 	 L = y’ - train_y	
5. 	 gra = 2*np.dot( (train_x)’ , L )
6. 	 weight vector -= learning rate * gra

In [23]:
# Add the bias part directly to the train_x to avoid matrix addition
train_x = np.concatenate((np.ones((train_x.shape[0],1)),train_x),axis=1)
print("train_x_shape : " , np.shape(train_x))
print("train_y_shape : " , np.shape(train_y))

# Initialize weight vector(including bias), learning rate and # of iteration
w = np.zeros(train_x.shape[1]) #  weight# = parameter#+1
print("w_shape : " , np.shape(w))
lrate = 10
regular = 0.1
itnum = 10000
epsilon = math.pow(10,-8)
totgra = np.zeros(w.shape)

for _ in range(itnum):
    yhat = np.dot(train_x,w)
    #print("yhat:" , yhat)
    diff = yhat - train_y
    #print("diff: " , diff)
    #print("diff_shape : " , np.shape(diff))
    cost = np.sum(diff**2) / len(train_x)
    cost1 = math.sqrt(cost)
    #print("cost1 : " , cost1)
    loss = np.sum(diff**2) + regular * np.sum(w ** 2)
    gra =  np.dot(np.transpose(train_x),diff)
    #print("gra : " , gra)
    totgra += gra**2
    #print("totgra: " , totgra)
    ada = np.sqrt(epsilon+totgra)
    #print("ada : " , ada)
    #w = w - lrate * np.divide( gra , ada )
    w = w*(1-lrate*regular/len(train_x)) - lrate * np.divide( gra , ada ) / len(train_x)
    #print("w: " , w)
    print ('iteration: %d | Cost: %f  ' % ( _,cost1))
print("diff_shape : " , np.shape(diff))
print("gra_shape : " , np.shape(gra))

train_x_shape :  (5652, 163)
train_y_shape :  (5652,)
w_shape :  (163,)
iteration: 0 | Cost: 27.071215  
iteration: 1 | Cost: 20.240154  
iteration: 2 | Cost: 17.467467  
iteration: 3 | Cost: 16.152382  
iteration: 4 | Cost: 15.498265  
iteration: 5 | Cost: 15.157447  
iteration: 6 | Cost: 14.966393  
iteration: 7 | Cost: 14.847188  
iteration: 8 | Cost: 14.762694  
iteration: 9 | Cost: 14.695221  
iteration: 10 | Cost: 14.636337  
iteration: 11 | Cost: 14.582014  
iteration: 12 | Cost: 14.530319  
iteration: 13 | Cost: 14.480313  
iteration: 14 | Cost: 14.431536  
iteration: 15 | Cost: 14.383753  
iteration: 16 | Cost: 14.336841  
iteration: 17 | Cost: 14.290727  
iteration: 18 | Cost: 14.245368  
iteration: 19 | Cost: 14.200731  
iteration: 20 | Cost: 14.156791  
iteration: 21 | Cost: 14.113528  
iteration: 22 | Cost: 14.070924  
iteration: 23 | Cost: 14.028961  
iteration: 24 | Cost: 13.987623  
iteration: 25 | Cost: 13.946896  
iteration: 26 | Cost: 13.906765  
iteration: 27 | Cost

# Step 4: Training finished! Save the training model

In [24]:
# save model
np.save('pm25model.npy',w)
# read model
w = np.load('pm25model.npy')

# Step 5: Load the testing data
* Each row: column 2~10 has data for the first 9 hours
* Every 18 row belongs to one data point (18 chemical)
* Totally the input variable is 18*9 = 162 (if add bias then 163) one chemical line up for nine hrs then another 
* Reorganize the testing data into N(=row number/18=4320/18=240) * 162

In [25]:
temp = []
# 每一個維度儲存一個“九小時的十八種化學物”資訊
text1 = open('test.csv', 'r', encoding='big5') 
row = csv.reader(text1 , delimiter=",")
n_row = 0
n_group = 0
for r in row:
    if n_row % 18 == 0 :
        n_group += 1
        temp.append([])
    for i in range(2,11):
        if r[i] != "NR":
            temp[n_group-1].append(float(r[i]))
        else:
            temp[n_group-1].append(float(0))
    n_row = n_row+1
text1.close()
print("temp_shape : " , np.shape(temp))
temp = np.array(temp)

temp_shape :  (240, 162)


# Step 6: Use the trained/save model to predict pm2.5
1. add another dimension on x for bias (162->163)
2. (Pseudo code)
3. read test_x.csv
4. every 18 rows : 
5. test_x.append([1])
6. test_x.append(這9小時的data)
7. test_y = np.dot( weight vector, test_x)

In [26]:
# Add the bias part directly to the train_x to avoid matrix addition
testx = np.concatenate((np.ones((temp.shape[0],1)),temp),axis=1)
print("testx_shape : " , np.shape(testx))

testx_shape :  (240, 163)


In [27]:
for _ in range(testx.shape[0]):
    testy = np.dot(testx,w)
print("testy_shape : " , np.shape(testy))
print(type(testy))
# add id for the first column of testy
testy = np.column_stack((np.arange(testy.shape[0])+1,testy))
testy = testy.tolist()
print("testy_shape : " , np.shape(testy))
print(type(testy))

testy_shape :  (240,)
<class 'numpy.ndarray'>
testy_shape :  (240, 2)
<class 'list'>


# Step 7: Output prediction to csv
* First Column is their id number
* Second Column is their pm2.5 forecast

In [28]:
filename = "predict.csv"
text = open(filename, "w+")
s = csv.writer(text,delimiter=',',lineterminator='\n')
s.writerow(["id","value"])
for i in range(len(testy)):
    s.writerow(testy[i]) 
text.close()

## Reference:
* Blog about gradient descent and Adagrad : http://cpmarkchang.logdown.com/posts/275500-optimization-method-adagrad
